In [1]:
import pandas as pd
import datetime


In [3]:
# read excel
df = pd.read_excel("data.xlsx")
df

,Date,Name,Task Type,Unique Identifier,Processing Time,Status,Task Count
0,2022-08-01,Juan,Inventory,XXX1,00:55:00,Pended,0
1,2022-08-01,Pedro,Admin,XXX2,00:46:00,Pended,0
2,2022-08-01,Santiago,CS,XXX3,00:35:00,Pended,0
3,2022-08-02,Juan,Inventory,XXX1,00:15:00,Pended,0
4,2022-08-02,Pedro,Admin,XXX2,00:23:00,Pended,0
5,2022-08-02,Santiago,CS,XXX3,00:10:00,Pended,0
6,2022-08-03,Juan,Inventory,XXX1,00:54:00,Completed,1
7,2022-08-03,Pedro,Admin,XXX2,00:43:00,Completed,1
8,2022-08-03,Santiago,CS,XXX3,00:31:00,Completed,1


In [5]:
df_sorted = df.sort_values(by=["Unique Identifier", "Date"])
df_sorted["Processing Time"] = pd.to_timedelta(df_sorted["Processing Time"].astype(str))

In [6]:
new_data = []
current_working_data = []
previous_identifier = ""
working_index = 0
column_names = ["Date", "Name", "Task Type", "Unique Identifier", "Processing Time", "Status", "Task Count"]
for index, row in df_sorted.reset_index().iterrows():
    if index == 0 and row["Status"].lower() == "completed":
        date = row["Date"]
        name = row["Name"]
        task_type = row["Task Type"]
        unique_identifier = row["Unique Identifier"]
        processing_time = row["Processing Time"]
        status = "Completed"
        task_count = 1

        data = pd.Series([date, name, task_type, unique_identifier, processing_time, status, task_count])
        new_data.append(data)
        continue
    if working_index == 0:
        current_working_data.append(row)
        previous_identifier = row["Unique Identifier"]
        working_index += 1
        continue

    current_identifier = row["Unique Identifier"]
    if current_identifier == previous_identifier:
        current_working_data.append(row)

        if row["Status"].lower() == "completed":
            date = current_working_data[-1]["Date"]
            name = current_working_data[-1]["Name"]
            task_type = current_working_data[-1]["Task Type"]
            unique_identifier = current_working_data[-1]["Unique Identifier"]

            processing_times = [item["Processing Time"] for item in current_working_data]
            processing_time = pd.Series(processing_times)
            total_processing_time = processing_time.sum()
            status = "Completed"
            task_count = 1

            data = pd.Series([date, name, task_type, unique_identifier, total_processing_time, status, task_count])
            new_data.append(data)

            previous_identifier = ""
            current_working_data = []
            working_index = 0
            continue

    else:
        for item in current_working_data:
            date = item["Date"]
            name = item["Name"]
            task_type = item["Task Type"]
            unique_identifier = item["Unique Identifier"]
            processing_time = item["Processing Time"]
            status = "Pended"
            task_count = 0

            data = pd.Series([date, name, task_type, unique_identifier, processing_time, status, task_count])
            new_data.append(data)

        previous_identifier = ""
        current_working_data = []
        working_index = 0

        current_working_data.append(row)
        previous_identifier = row["Unique Identifier"]
        working_index += 1
        continue

    working_index += 1
    previous_identifier = row["Unique Identifier"]

new_df = pd.DataFrame(new_data)
new_df.columns = column_names
new_df_sorted = new_df.sort_values(["Date", "Unique Identifier"])
new_df_sorted["Processing Time"] = new_df_sorted["Processing Time"].astype(str).str[-8:]
new_df_sorted


,Date,Name,Task Type,Unique Identifier,Processing Time,Status,Task Count
0,2022-08-03,Juan,Inventory,XXX1,02:04:00,Completed,1
1,2022-08-03,Pedro,Admin,XXX2,01:52:00,Completed,1
2,2022-08-03,Santiago,CS,XXX3,01:16:00,Completed,1


In [8]:
new_df_sorted["Date"] = pd.to_datetime(new_df_sorted["Date"]).dt.date
new_df_sorted.to_excel("output.xlsx", index=False)
